In [195]:
import tensorflow as tf
import pandas as pd
import numpy as np
import kerastuner as kt
from kerastuner import RandomSearch
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from datetime import datetime
from keras.callbacks import EarlyStopping
pd.set_option('display.max_columns', 500)
from sklearn.feature_selection import SelectKBest, chi2, f_classif
import math

In [3]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [239]:
df = pd.read_csv('for_tf_df.csv', index_col = [0]).reset_index(drop = True)
df = df[df['odds_sum'] > 1.02]
df = df.dropna(axis = 1)
print(df.shape)

(358, 334)


In [240]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:, -1], test_size = 0.25)

In [241]:
X_train_n = tf.keras.utils.normalize(X_train, axis = 1)
X_test_n = tf.keras.utils.normalize(X_test, axis = 1)

In [252]:
def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 4
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=5, max_value=15, step=2),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=3, activation="softmax"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [272]:
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     executions_per_trial = 1,
                     factor = 3,
                     directory="kt_dir",
                     project_name="kt_hyperband",)
early_stopping = EarlyStopping(monitor="loss", patience= 10, verbose=0)

INFO:tensorflow:Reloading Oracle from existing project kt_dir\kt_hyperband\oracle.json


In [ ]:
tuner.search(X_train_n, y_train, epochs=150, validation_data = (X_test_n, y_test), callbacks=[early_stopping], verbose=1)

In [260]:
best_hps=tuner.get_best_hyperparameters()[0]
h_model = tuner.hypermodel.build(best_hps)
h_model.fit(X_train_n, y_train, epochs=150, callbacks=[early_stopping], verbose=1)


Epoch 1/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0548 - accuracy: 0.3881
Epoch 2/150
9/9 [==============================] - 0s 1ms/step - loss: 0.9743 - accuracy: 0.5784
Epoch 3/150
9/9 [==============================] - 0s 2ms/step - loss: 0.9166 - accuracy: 0.6119
Epoch 4/150
9/9 [==============================] - 0s 2ms/step - loss: 0.8745 - accuracy: 0.6194
Epoch 5/150
9/9 [==============================] - 0s 1ms/step - loss: 0.8455 - accuracy: 0.6194
Epoch 6/150
9/9 [==============================] - 0s 2ms/step - loss: 0.8261 - accuracy: 0.6343
Epoch 7/150
9/9 [==============================] - 0s 2ms/step - loss: 0.8094 - accuracy: 0.6493
Epoch 8/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7984 - accuracy: 0.6567
Epoch 9/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7815 - accuracy: 0.6530
Epoch 10/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7723 - accuracy: 0.6679
Epoch 11/150
9/9 [===========

9/9 [==============================] - 0s 2ms/step - loss: 0.4026 - accuracy: 0.8545
Epoch 85/150
9/9 [==============================] - 0s 2ms/step - loss: 0.4003 - accuracy: 0.8507
Epoch 86/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3958 - accuracy: 0.8545
Epoch 87/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3920 - accuracy: 0.8433
Epoch 88/150
9/9 [==============================] - 0s 1ms/step - loss: 0.3889 - accuracy: 0.8545
Epoch 89/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3857 - accuracy: 0.8507
Epoch 90/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3817 - accuracy: 0.8545
Epoch 91/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3819 - accuracy: 0.8507
Epoch 92/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3779 - accuracy: 0.8545
Epoch 93/150
9/9 [==============================] - 0s 2ms/step - loss: 0.3747 - accuracy: 0.8619
Epoch 94/150
9/9 [===============

In [262]:
h_model.evaluate(X_test_n, y_test)

3/3 [==============================] - 0s 2ms/step - loss: 1.2564 - accuracy: 0.5889


[1.256432056427002, 0.5888888835906982]

In [246]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(7, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(7, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))
es = EarlyStopping(monitor='accuracy', mode='min', verbose=1)
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(x = X_train_n, y= y_train,  epochs = 150)

Epoch 1/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0924 - accuracy: 0.5000
Epoch 2/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0880 - accuracy: 0.5037
Epoch 3/150
9/9 [==============================] - 0s 1ms/step - loss: 1.0827 - accuracy: 0.5336
Epoch 4/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0778 - accuracy: 0.5187
Epoch 5/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0727 - accuracy: 0.5336
Epoch 6/150
9/9 [==============================] - 0s 1ms/step - loss: 1.0681 - accuracy: 0.5485
Epoch 7/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0630 - accuracy: 0.5560
Epoch 8/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0578 - accuracy: 0.5485
Epoch 9/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0534 - accuracy: 0.5485
Epoch 10/150
9/9 [==============================] - 0s 2ms/step - loss: 1.0481 - accuracy: 0.5485
Epoch 11/150
9/9 [===========

9/9 [==============================] - 0s 2ms/step - loss: 0.7553 - accuracy: 0.6716
Epoch 85/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7542 - accuracy: 0.6754
Epoch 86/150
9/9 [==============================] - 0s 1ms/step - loss: 0.7519 - accuracy: 0.6716
Epoch 87/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7522 - accuracy: 0.6716
Epoch 88/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7488 - accuracy: 0.6791
Epoch 89/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7477 - accuracy: 0.6754
Epoch 90/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7463 - accuracy: 0.6679
Epoch 91/150
9/9 [==============================] - 0s 1ms/step - loss: 0.7450 - accuracy: 0.6679
Epoch 92/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7421 - accuracy: 0.6754
Epoch 93/150
9/9 [==============================] - 0s 2ms/step - loss: 0.7405 - accuracy: 0.6679
Epoch 94/150
9/9 [===============

In [247]:
model.evaluate(X_test_n, y_test)

3/3 [==============================] - 0s 2ms/step - loss: 0.8641 - accuracy: 0.5889


[0.8641465902328491, 0.5888888835906982]

In [263]:
preds = pd.DataFrame(h_model.predict(X_test_n), columns = ['pred 0','pred 1','pred 2'])
predicted_df = pd.concat([X_test.reset_index(drop = True), preds], axis = 1)
# bo_po_dha = list(zip(predicted_df[['med_0','med_1','med_2']],predicted_df[['pred 0','pred 1','pred 2']],['draw','home','away']))
bo_po_dha = list(zip(predicted_df[['75_0','75_1','75_2']],predicted_df[['pred 0','pred 1','pred 2']],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 0.25), axis=1)
predicted_df['target'] = y_test.reset_index(drop=True)
dha = {'draw':'0','home':'1','away':'2'}
predicted_df = predicted_df.replace('no value', 0)
predicted_df[['draw','home','away']] = predicted_df[['draw','home','away']].astype(float)
for word, number in dha.items():
    predicted_df[f'{word}_profits'] = np.where(predicted_df['target'] == int(number),
                                    predicted_df[word] * (predicted_df['75_'+ number] - 1) * 100,
                                    -100*predicted_df[word])

predicted_df['profits'] = predicted_df[predicted_df.columns[-3:]].sum(axis = 1)
p = round(np.sum(predicted_df["profits"]),3)
predicted_df = predicted_df.round(4)
print(f'number of bets made: {predicted_df.shape[0]}\n')
print(f'profit on $100 bankroll: ${p}\n')
print(predicted_df.sum()[-4:-1])

3/3 [==============================] - 0s 3ms/step
number of bets made: 90

profit on $100 bankroll: $8.923

draw_profits    -89.5305
home_profits    100.8850
away_profits     -2.4312
dtype: float64


In [117]:
checker = predicted_df[['score_diff','med_0','med_1','med_2','odds_sum','pred 0','pred 1',
              'pred 2','target','draw','home','away','draw_profits','home_profits','away_profits','profits']]

In [118]:
odds_compare = checker[['med_0','med_1','med_2']]
odds_compare[['predo_0','predo_1','predo_2']] = checker[['pred 0','pred 1','pred 2']].apply(lambda x: 1/x)

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [119]:
odds_compare.head(40)

,med_0,med_1,med_2,predo_0,predo_1,predo_2
0,5.5000,1.2000,13.0000,7.057163,1.237164,20.000000
1,3.5000,11.0000,1.4444,4.048583,17.301039,1.438228
2,7.0000,1.1333,25.0000,8.952552,1.169180,30.395136
3,3.8500,1.4000,8.5000,5.385030,1.372684,11.655012
4,19.0000,1.0250,51.0000,11.750881,1.117693,49.504951
5,2.4500,2.6000,3.6000,3.266906,4.859087,2.048760
6,3.9000,1.4444,8.0000,5.005005,1.434309,9.708738
7,2.7000,2.9000,2.7500,3.325573,3.205128,2.582645
8,4.2250,1.3417,9.0000,5.608525,1.350986,12.269938
9,3.2500,1.5714,7.5000,44.642860,1.024905,526.315796
